In [2]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical

Using TensorFlow backend.


In [ ]:
df = pd.read_csv("/home/ec2-user/Data/restuarant_review_balanced.csv")
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)
train.drop('stars',axis = 1,inplace = True)
test.drop('stars',axis = 1,inplace = True)
positive_train = train[train['sentiment'] == 1]
negative_train = train[train['sentiment'] == 0]
print(positive_train.shape)
print(negative_train.shape)
positive_test = test[test['sentiment'] == 1]
negative_test = test[test['sentiment'] == 0]
print(positive_test.shape)
print(negative_test.shape)

In [15]:
train.head()

,sentiment,Processed_Reviews
151246,0,much better chicken place go days theyre slow ...
87000,0,place overrate burger dry stingy gravy poutine...
102233,1,bbq probably favorite food give usually pricey...
254520,0,food service restaurant leave much desire arri...
135806,0,chicken red curry lunch special piece chicken ...


(109668, 2)
(96860, 2)
(27507, 2)
(24125, 2)


In [17]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])

maxlen = 130
x_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
y_train = train['sentiment']

y_test = test["sentiment"]
list_sentences_test = test['Processed_Reviews']
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
x_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [18]:
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size,input_length = maxlen))
model.add(Conv1D(filters = 50, kernel_size = 2, padding='same',activation = "relu"))
model.add(GlobalMaxPool1D())
#model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))
model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 128)          768000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 130, 50)           12850     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 780,901
Trainable params: 780,901
Non-trainable params: 0
_________________________________________________________________


In [21]:
batch_size = 512
epochs = 10
model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test))

Train on 206528 samples, validate on 51632 samples
Epoch 1/10
206528/206528 [==============================] - 8s 41us/step - loss: 0.0528 - acc: 0.9838 - val_loss: 0.1480 - val_acc: 0.9470
Epoch 2/10
206528/206528 [==============================] - 8s 40us/step - loss: 0.0433 - acc: 0.9878 - val_loss: 0.1531 - val_acc: 0.9470
Epoch 3/10
206528/206528 [==============================] - 8s 40us/step - loss: 0.0370 - acc: 0.9903 - val_loss: 0.1592 - val_acc: 0.9464
Epoch 4/10
206528/206528 [==============================] - 8s 40us/step - loss: 0.0312 - acc: 0.9924 - val_loss: 0.1665 - val_acc: 0.9460
Epoch 5/10
206528/206528 [==============================] - 8s 40us/step - loss: 0.0258 - acc: 0.9942 - val_loss: 0.1758 - val_acc: 0.9455
Epoch 6/10
206528/206528 [==============================] - 8s 40us/step - loss: 0.0209 - acc: 0.9959 - val_loss: 0.1858 - val_acc: 0.9446
Epoch 7/10
206528/206528 [==============================] - 8s 40us/step - loss: 0.0165 - acc: 0.9970 - val_loss: 0

KeyboardInterrupt: 

51632
51632


0
0


accuracy :0.9454795475674
F1-score: 0.9489453542992907
Confusion matrix:
